***
__ClassFractions__ fractions (frequency) image for class values in as  ImageCollection of classifications
***

In [ ]:
import geemap
import ee
if not ee.data._credentials: ee.Initialize()

import sys
sys.path.append('../src/geepatches')
import geeutils
import geeproduct
import geemask

__ClassFractions S2 Scene CLassification__ 
- are cloud-related classes homogeneous?
- 'bright' buildigs are abnormally 'clouded'
- water (rivers) seems abnormally 'clouded'

In [ ]:
eepoint            = geeutils.half31UESpoint #geeutils.tennvenlopoint #geeutils.brusselspoint
eedatefrom         = geeutils.half31UESday   #ee.Date('2018-01-01') 
eedatetill         = eedatefrom.advance(1, 'year')
eeroi              = geeutils.squareareaboundsroi(eepoint, 25000)
verbose            = False

#
# background rgb
#
rgbimage = geeproduct.GEECol_s2rgb().collect(eeroi, geeutils.clearskyday,  geeutils.clearskyday.advance(1, 'day')).first()

#
# scl collection
#
sclimagecollection = geeproduct.GEECol_s2scl().collect(eeroi, eedatefrom, eedatetill, verbose=verbose)
sclimage           = sclimagecollection.first()

#
# class fractions
#
classfractionsmaker = geemask.ClassFractions([3,8,9,10,11])
fractionsimage      = classfractionsmaker.makefractions(sclimagecollection)
fractionsimage      = fractionsimage.setDefaultProjection(sclimagecollection.first().projection())

print (geeutils.szestimatevaluesinfo(fractionsimage.clip(eeroi))) # clip; otherwise unbounded
xin = fractionsimage.clip(eeroi).reduceRegion(ee.Reducer.min()).values().get(0).getInfo()
xax = fractionsimage.clip(eeroi).reduceRegion(ee.Reducer.max()).values().get(0).getInfo()


map = geemap.Map(height='600px')
map.centerObject(eepoint, 13)
map.addLayer(rgbimage,                                       {'min': 1,  'max':255},                 'rgb')
map.addLayer(sclimage,                                       geeutils.s2sclvisParams,                'scl')

map.addLayer(fractionsimage,                                 {'min':xin,  'max':xax},                'fct')                        
map.addLayer(geeutils.outlinegeometryimage(eepoint, 100, 2), {'palette':'#ff0000'},                  'roi')
map

__ClassFractions S2 Scene CLassification__ 
- water (rivers) seems abnormally 'dark area' too

In [ ]:
eepoint            = geeutils.half31UESpoint #geeutils.tennvenlopoint #geeutils.brusselspoint
eedatefrom         = geeutils.half31UESday   #ee.Date('2018-01-01') 
eedatetill         = eedatefrom.advance(1, 'year')
eeroi              = geeutils.squareareaboundsroi(eepoint, 25000)
verbose            = False

rgbimage = geeproduct.GEECol_s2rgb().collect(eeroi, geeutils.clearskyday,  geeutils.clearskyday.advance(1, 'day')).first()

sclimagecollection = geeproduct.GEECol_s2scl().collect(eeroi, eedatefrom, eedatetill, verbose=verbose)

map = geemap.Map(height='600px')
map.centerObject(eepoint, 13)
map.addLayer(rgbimage, {'min': 1,  'max':255}, 'rgb')

for scclass in [2,4,5,6,7] :
    classfractionsmaker = geemask.ClassFractions([scclass])
    fractionsimage      = classfractionsmaker.makefractions(sclimagecollection)
    fractionsimage      = fractionsimage.setDefaultProjection(sclimagecollection.first().projection())

    print (f"class({scclass})\n {geeutils.szestimatevaluesinfo(fractionsimage.clip(eeroi))}") # clip; otherwise unbounded
    xin = fractionsimage.clip(eeroi).reduceRegion(ee.Reducer.min()).values().get(0).getInfo()
    xax = fractionsimage.clip(eeroi).reduceRegion(ee.Reducer.max()).values().get(0).getInfo()


    map.addLayer(fractionsimage, {'min':xin,  'max':xax}, f"fct({scclass})")                        

map.addLayer(geeutils.outlinegeometryimage(eepoint, 100, 2), {'palette':'#ff0000'},                  'roi')
map

In [ ]:
eepoint            = geeutils.half31UESpoint #geeutils.tennvenlopoint #geeutils.brusselspoint
eedatefrom         = geeutils.half31UESday   #ee.Date('2018-01-01') 
eedatetill         = eedatefrom.advance(1, 'year')
eeroi              = geeutils.squareareaboundsroi(eepoint, 25000)
verbose            = False

rgbimage = geeproduct.GEECol_s2rgb().collect(eeroi, geeutils.clearskyday,  geeutils.clearskyday.advance(1, 'day')).first()

sclimagecollection = geeproduct.GEECol_s2scl().collect(eeroi, eedatefrom, eedatetill, verbose=verbose)

map = geemap.Map(height='600px')
map.centerObject(eepoint, 13)
map.addLayer(rgbimage, {'min': 1,  'max':255}, 'rgb')

for scclass in [2,4,5,6,7] :
    classfractionsmaker = geemask.ClassFractions([scclass], [3,8,9,10,11])
    fractionsimage      = classfractionsmaker.makefractions(sclimagecollection)
    fractionsimage      = fractionsimage.setDefaultProjection(sclimagecollection.first().projection())

    print (f"class({scclass})\n {geeutils.szestimatevaluesinfo(fractionsimage.clip(eeroi))}") # clip; otherwise unbounded
    xin = fractionsimage.clip(eeroi).reduceRegion(ee.Reducer.min()).values().get(0).getInfo()
    xax = fractionsimage.clip(eeroi).reduceRegion(ee.Reducer.max()).values().get(0).getInfo()


    map.addLayer(fractionsimage, {'min':xin,  'max':xax}, f"fct({scclass})")                        

map.addLayer(geeutils.outlinegeometryimage(eepoint, 100, 2), {'palette':'#ff0000'},                  'roi')
map